In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D, Flatten,Dense
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import numpy as np
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
/content/drive/My Drive/Yarisma/Data2


In [37]:
# Temizlenmiş Birleştirilmiş Veri setlerini oku
data = pd.read_csv("")
print("sütun sayısı:",data.shape[1])
print("satır sayısı:",data.shape[0])

sütun sayısı: 199
satır sayısı: 46944


In [ ]:
data.head()

In [38]:
# Tarih sütununu datetime nesnelerine dönüştür
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

# Tarih sütununu 3 ayrı sütuna ayırma
data['Yıl'] = data['date'].dt.year
data['Ay'] = data['date'].dt.month
data['Gün'] = data['date'].dt.day
data.drop(columns=['date'],inplace=True)
#data.drop(columns=['Bayram_Flag'],inplace=True)

print("sütun sayısı:",data.shape[1])
print("satır sayısı:",data.shape[0])

sütun sayısı: 201
satır sayısı: 46944


In [39]:
data.drop(columns=[''],inplace=True)

In [40]:
# Kategorik değişkenleri one-hot encoding ile dönüştür
categorical_features = ['']
data = pd.get_dummies(data, columns=categorical_features, dummy_na=True)

In [ ]:
# Label Encoder oluşturun
label_encoder = LabelEncoder()

# ilce sütununu Label Encoding yapın
data[''] = label_encoder.fit_transform(data[''])

In [41]:
# Bağımsız değişkenler ve bağımlı değişkenin ayrılması
X = data.drop(columns=[''])
y = data['']

In [42]:
# Eğitim ve test verilerinin bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [43]:
# MinMaxScaler'ı kullanarak verileri normalize etme
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)


In [ ]:
print(X_train_normalized)

In [44]:
# Modeli oluştur
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mae')

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 244, 64)           256       
                                                                 
 conv1d_6 (Conv1D)           (None, 242, 64)           12352     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 121, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, 119, 128)          24704     
                                                                 
 conv1d_8 (Conv1D)           (None, 117, 128)          49280     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 58, 128)           0         
 g1D)                                                 

In [30]:
# Önceden eğitilmiş ağırlıkları yükleyin
model.load_weights('best_model_weights.h5')

# Modeli derleme
model.compile(optimizer='adam', loss='mae')

In [46]:
# Veriyi CNN modeline uygun hale getir
X_train_normalized = X_train_normalized.reshape((X_train_normalized.shape[0], X_train_normalized.shape[1], 1))
X_test_normalized = X_test_normalized.reshape((X_test_normalized.shape[0], X_test_normalized.shape[1], 1))


In [ ]:
X_test_normalized.shape

In [47]:
# X_train_cnn ve X_test_cnn verilerinin dtype'ını dönüştür
X_train_normalized = X_train_normalized.astype('float64')
X_test_normalized = X_test_normalized.astype('float64')

In [48]:
checkpoint = ModelCheckpoint("minmaxscaler_weights.h5",
                             monitor='val_loss',  # Kaydetme kriteri olarak doğruluk metriğini kullanın
                             verbose=1,
                             save_best_only=True,    # Sadece en iyi performansı gösteren ağırlıkları kaydedin
                             mode='min')
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, mode='auto', factor=0.1, min_lr=0.000001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [49]:
# Modeli eğit
history=model.fit(X_train_normalized, y_train, epochs=100, batch_size=32,validation_split=0.2, verbose=1,callbacks=[reduce_lr,checkpoint,early_stopping])

Epoch 1/100
939/939 [==============================] - ETA: 0s - loss: 3.0825
Epoch 1: val_loss improved from inf to 2.97101, saving model to minmaxscaler_weights(DENEME2).h5
939/939 [==============================] - 117s 123ms/step - loss: 3.0825 - val_loss: 2.9710 - lr: 0.0010
Epoch 2/100
  1/939 [..............................] - ETA: 1:21 - loss: 3.5359

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


939/939 [==============================] - ETA: 0s - loss: 2.7973
Epoch 2: val_loss improved from 2.97101 to 2.73728, saving model to minmaxscaler_weights(DENEME2).h5
939/939 [==============================] - 111s 119ms/step - loss: 2.7973 - val_loss: 2.7373 - lr: 0.0010
Epoch 3/100
939/939 [==============================] - ETA: 0s - loss: 2.7271
Epoch 3: val_loss did not improve from 2.73728
939/939 [==============================] - 105s 112ms/step - loss: 2.7271 - val_loss: 2.7554 - lr: 0.0010
Epoch 4/100
939/939 [==============================] - ETA: 0s - loss: 2.6803
Epoch 4: val_loss improved from 2.73728 to 2.70263, saving model to minmaxscaler_weights(DENEME2).h5
939/939 [==============================] - 113s 120ms/step - loss: 2.6803 - val_loss: 2.7026 - lr: 0.0010
Epoch 5/100
939/939 [==============================] - ETA: 0s - loss: 2.6426
Epoch 5: val_loss improved from 2.70263 to 2.64816, saving model to minmaxscaler_weights(DENEME2).h5
939/939 [=======================

In [ ]:
# Modelin performansını değerlendirme (MAE)
y_pred = model.predict(X_test_normalized)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

In [ ]:
y_pred

# **Hibritlemek**

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train_normalized, y_train)

In [ ]:
rf_pred = rf_model.predict(X_test_normalized)
y_pred = model.predict(X_test_cnn)

In [ ]:
# Stacking için yeni bir özellik oluşturma (tahminlerin birleştirilmesi)
stacked_features = np.column_stack((rf_pred, y_pred))

# Meta-modelin oluşturulması
meta_model = RandomForestRegressor()
meta_model.fit(stacked_features, y_test)

In [ ]:
# Meta-model ile tahminlerin yapılması
stacked_pred = meta_model.predict(stacked_features)

# Sonuçların değerlendirilmesi
mae_rf = mean_absolute_error(y_test, rf_pred)
mae_lr = mean_absolute_error(y_test, y_pred)
mae_stacked = mean_absolute_error(y_test, stacked_pred)

print("Random Forest MAE:", mae_rf)
print("CNN MAE:", mae_lr)
print("Stacked Model MAE:", mae_stacked)

In [ ]:
# Tahminlerin orijinal ölçeğe dönüştürülmesi
predictions_original_scale = scaler.inverse_transform(predictions.reshape(-1, 1))


# **Tahmin Etme**

In [ ]:
# Test için hazırlanmış verileri oku
test_data = pd.read_csv("")

In [ ]:
# Tarih sütununu datetime nesnelerine dönüştür
test_data['date'] = pd.to_datetime(test_data['date'], format='%Y-%m-%d')

# Tarih sütununu 3 ayrı sütuna ayırma
test_data['Yıl'] = test_data['date'].dt.year
test_data['Ay'] = test_data['date'].dt.month
test_data['Gün'] = test_data['date'].dt.day
test_data.drop(columns=['date'],inplace=True)
test_data.drop(columns=['Bayram_Flag'],inplace=True)

print("sütun sayısı:",test_data.shape[1])
print("satır sayısı:",test_data.shape[0])

sütun sayısı: 201
satır sayısı: 1363


In [ ]:
temp_df = pd.DataFrame(test_data['ilce'], columns=['ilce'])


In [ ]:

# One-hot encoding işlemi
test_data = pd.get_dummies(test_data, columns=categorical_features, dummy_na=True)

# One-hot encoding sonrası oluşan sütunların isimlerini al
one_hot_encoded_columns_test = test_data.columns[test_data.columns.str.startswith(tuple(categorical_features))]


In [ ]:
test_data.drop(columns=['uniquie_id','tarih'],inplace=True)

In [ ]:
test_data.columns

In [ ]:
# Tahmin edilecek veriyi hazırla
X_test = test_data

In [ ]:
# Veriyi CNN modeline uygun hale getir
X_test_cnn = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
X_test_cnn = X_test_cnn.astype('float64')

In [ ]:
# Tahminlerin yapılması
rf_pred = rf_model.predict(X_test)
Cnn_pred = model.predict(X_test_cnn)

# Stacking için yeni bir özellik oluşturma (tahminlerin birleştirilmesi)
stacked_features = np.column_stack((rf_pred, Cnn_pred))

# Meta-model ile tahminlerin yapılması
predicted_values = meta_model.predict(stacked_features)

43/43 [==============================] - 1s 25ms/step


In [ ]:

# Tahmin edilen değerleri yeni bir sütun olarak test verisine ekle
test_data_with_predictions = X_test.copy()
test_data_with_predictions['Predicted_Values'] = predicted_values

# Tahmin edilen değerleri içeren DataFrame'i göster
test_data_with_predictions

In [ ]:
# Tarih birleştirme
test_data_with_predictions['Tarih'] = test_data['Yıl'].astype(str) + '-' + test_data['Ay'].astype(str).str.zfill(2) + '-' + test_data['Gün'].astype(str).str.zfill(2)

# ilçe ve tarih sütununu birleştirme, unique_id yapma
test_data_with_predictions['unique_id'] = test_data_with_predictions['Tarih'] + '-' + temp_df['ilce']

# Yeni bir DataFrame oluşturma
submission_df = pd.DataFrame()

# unique_id sütununu ekleme
submission_df['unique_id'] = test_data_with_predictions['unique_id']

# Bildirimsiz sum (tahmin değerleri) sütununu ekleme
submission_df['bildirimsiz_sum'] = test_data_with_predictions['Predicted_Values']

# Veriyi CSV olarak kaydetme
submission_df.to_csv('tahmin_rfr_cnn.csv', index=False)

# **Saf CNN ile tahmin etme**

In [31]:
# Test için hazırlanmış verileri oku
test_data = pd.read_csv("")

In [32]:
# Tarih sütununu datetime nesnelerine dönüştür
test_data['date'] = pd.to_datetime(test_data['date'], format='%Y-%m-%d')

# Tarih sütununu 3 ayrı sütuna ayırma
test_data['Yıl'] = test_data['date'].dt.year
test_data['Ay'] = test_data['date'].dt.month
test_data['Gün'] = test_data['date'].dt.day
test_data.drop(columns=['date'],inplace=True)


print("sütun sayısı:",test_data.shape[1])
print("satır sayısı:",test_data.shape[0])

sütun sayısı: 202
satır sayısı: 1363


In [33]:
temp_df = pd.DataFrame(test_data['ilce'], columns=['ilce'])


In [34]:

# One-hot encoding işlemi
test_data = pd.get_dummies(test_data, columns=categorical_features, dummy_na=True)

# One-hot encoding sonrası oluşan sütunların isimlerini al
one_hot_encoded_columns_test = test_data.columns[test_data.columns.str.startswith(tuple(categorical_features))]


In [35]:
# Tahmin edilecek veriyi hazırla
X_test = test_data.drop(columns=[''])

KeyError: "['bildirimsiz_sum'] not found in axis"

In [ ]:
X_test_normalized = scaler.transform(X_test)

In [ ]:
# Veriyi CNN modeline uygun hale getir
X_test_cnn = X_test_normalized.reshape((X_test_normalized.shape[0], X_test_normalized.shape[1], 1))

In [ ]:
# Test verisi üzerinde tahmin yap
predicted_values_cnn = model.predict(X_test_cnn)
print("Predicted Values (CNN):", predicted_values_cnn)

In [ ]:
# Test verilerini kullanarak tahmin yapın
predictions = model.predict(X_test_normalized)

# Tahminlerinizi gerçek ölçeklere dönüştürün
predicted_values = scaler.inverse_transform(predictions.reshape(-1, 1))

In [ ]:
# Tahmin edilen değerleri yeni bir sütun olarak test verisine ekle
test_data_with_predictions = X_test.copy()
test_data_with_predictions['Predicted_Values'] = predicted_values_cnn

# Tahmin edilen değerleri içeren DataFrame'i göster
test_data_with_predictions